### Retrieve global coins blockchain data

In [ ]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockchain data:')
# print(df)
global_df

Global blockchain data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,561.0000,3.125000,3.463612e-03,SHA-256,3.123667,154.010695,0.010824,1.666979,0.533661
BTC,576.0000,3.149835,1.000000e+00,SHA-256,851.292872,150.000000,3.149835,472.475284,0.555009
DOGE,63.0000,10000.000000,2.670000e-06,Scrypt,0.003086,1371.428571,0.026700,36.617143,11865.793982
KAS,1.0011,65.498412,1.019957e-06,kHeavyHash,1.365668,86305.064429,0.000067,5.765657,4.221858
LTC,158.0000,6.250000,1.263880e-03,Scrypt,0.002075,546.835443,0.007899,4.319590,2081.607827
RVN,59.0000,2500.000000,1.522035e-07,KawPow,0.000007,1464.406780,0.000381,0.557220,82663.680010


### Retrieve NiceHash marketplace algorithms data

In [1]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

NameError: name 'btc_exchange_rate' is not defined

### Calculate Nicehash max daily and max hourly consumption rate

In [ ]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,1.007775e-04,1.659020,0.069126,"$6,649.68"
SHA256ASICBOOST,5.303328e+00,2.873189,0.119716,"$11,516.30"
KAWPOW,5.500135e-07,0.052144,0.002173,$209.00
KHEAVYHASH,4.110375e-02,0.166365,0.006932,$666.82


### Calculate Nicehash hashrate to global hashrate ratio
Cap Nicehash hashrate to 15% of global hashrate

In [ ]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']
reachable_df['nh_to_global_ratio_capped'] = reachable_df['nh_hashrate_eh_s_capped'] / reachable_df['nethash_eh_s']  

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped,nh_to_global_ratio_capped
BCH,SHA-256,3.123667,5.303328e+00,1.697789,4.685500e-01,0.150000
BTC,SHA-256,851.292872,5.303328e+00,0.006230,5.303328e+00,0.006230
DOGE,Scrypt,0.003086,1.007775e-04,0.032657,1.007775e-04,0.032657
KAS,kHeavyHash,1.365668,4.110375e-02,0.030098,4.110375e-02,0.030098
LTC,Scrypt,0.002075,1.007775e-04,0.048565,1.007775e-04,0.048565
RVN,KawPow,0.000007,5.500135e-07,0.081595,5.500135e-07,0.081595


### Calculate burn rate needed for specific amount to be spent in the time period
```
B - burn rate
A - amount
P - price
D - duration

B = A / P * D
```

In [ ]:
min_order_amount_btc = 0.0001 # BTC
order_duration_s = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['amount_btc'] = min_order_amount_btc
speed_df['duration_s'] = order_duration_s
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = speed_df.amount_btc / (speed_df.paying_btc_eh_s * speed_df.duration_s)
speed_df

,amount_btc,duration_s,paying_btc_eh_s,speed_eh_s
SCRYPT,0.0001,3600,0.190535,1.457884e-07
SHA256ASICBOOST,0.0001,3600,0.000006,4.429917e-03
KAWPOW,0.0001,3600,1.097272,2.531530e-08
KHEAVYHASH,0.0001,3600,0.000047,5.929661e-04


### Calculate amount needed to buy speed for the time period
```
B - burn rate
A - amount
P - price
D - duration

A = B * P * D 
```

In [ ]:
speed_eh_s = 1 # EH / s
order_duration_s = 3600  # 1 hour

amount_df = pd.DataFrame(index=nh_market_df.index)
amount_df['speed_eh_s'] = speed_eh_s
amount_df['duration_s'] = order_duration_s
amount_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
amount_df['amount_btc'] = amount_df.speed_eh_s * amount_df.paying_btc_eh_s * amount_df.duration_s
amount_df

,speed_eh_s,duration_s,paying_btc_eh_s,amount_btc
SCRYPT,1,3600,0.190535,685.925482
SHA256ASICBOOST,1,3600,0.000006,0.022574
KAWPOW,1,3600,1.097272,3950.180009
KHEAVYHASH,1,3600,0.000047,0.168644


### Calculate RTP for the coin on the NiceHash market

In [ ]:
import pandas as pd
import numpy as np

max_allowed_coin_market_hp_share = 0.15
max_nicehash_market_utilisation = 0.76
speed_eh_s = 1
order_duration_s = 3600
order_amount_btc = 1

rtp_df = pd.DataFrame()
# rtp_df.loc['BCH', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BCH', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['BTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['DOGE', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['DOGE', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['LTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['LTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['KAS', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KHEAVYHASH', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['KAS', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['RVN', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KAWPOW', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['RVN', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df['nh_to_global_ratio'] = rtp_df['nh_hashrate_eh_s'] / global_df['nethash_eh_s']
# rtp_df['max_global_btc_reward_h'] = rtp_df.nh_to_global_ratio * global_df['daily_emmisions_btc'] / 24
# rtp_df['max_global_btc_reward_h'] = global_df['daily_emmisions_btc'] / 24
rtp_df['global_price_btc_eh_s'] = global_df['index_price_btc_eh_day'] / 86400
rtp_df.loc['BCH', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['BTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['DOGE', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
rtp_df.loc['LTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
rtp_df.loc['KAS', 'nh_price_btc_eh_s'] = nh_market_df.loc['KHEAVYHASH', 'paying_btc_eh_day'] / 86400
rtp_df.loc['RVN', 'nh_price_btc_eh_s'] = nh_market_df.loc['KAWPOW', 'paying_btc_eh_day'] / 86400
rtp_df['global_1eh_1h_duration_reward_btc'] = rtp_df.global_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['nh_1eh_1h_duration_expense_btc'] = rtp_df.nh_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['rtp_%'] =  rtp_df.global_1eh_1h_duration_reward_btc / rtp_df.nh_1eh_1h_duration_expense_btc * 100
rtp_df['allocation_%'] = rtp_df['rtp_%'] / rtp_df['rtp_%'].sum() * 100

rtp_df


,global_price_btc_eh_s,nh_price_btc_eh_s,global_1eh_1h_duration_reward_btc,nh_1eh_1h_duration_expense_btc,rtp_%,allocation_%
BCH,0.000006,0.000006,0.022236,0.022574,98.503064,20.643032
BTC,0.000006,0.000006,0.023125,0.022574,102.443501,21.468819
DOGE,0.137336,0.190535,494.408083,685.925482,72.078979,15.105405
KAS,0.000049,0.000047,0.175911,0.168644,104.309121,21.859792
LTC,0.024093,0.190535,86.733659,685.925482,12.644764,2.649930
RVN,0.956756,1.097272,3444.320000,3950.180009,87.194001,18.273021
